### Assignment 3

Implement the “Self Alignment with Instruction Backtranslation” paper. When fine tuning the model, use LoRA. You will not be able to do full finetuning because there is not enough memory.

Link to paper: https://arxiv.org/pdf/2308.06259.pdf
Colab’s GPU usage is limited. Try to first prototype and get things working on the CPU first before training on the GPU with the full dataset. If you are not able to connect to a GPU on colab, you can try to create a PyTorch Lightning Studio or a Kaggle notebook.

In particular:

1. Finetune the base language model (llama2 7B) with (output, instruction) pairs {(yi, xi)} from the seed data to obtain a backward model Myx := p(x|y). In other words, finetune a model that uses the output to predict the instruction. Use the openassistant-guanaco training set dataset. (25 points).

- Push the backwards model to HF and paste url here

 https://huggingface.co/Justin8584/backward_model

2. Self-Augmentation -- Randomly sample a subset of size 150 and generate instructions from the LIMA dataset’s completions and filtering out any mutli-turn examples. Print out 5 examples of generated instructions. (25 points)

- (generated instructions from backwards model, response is from LIMA) pairs
- Single turn:
-- Single turn: (What is the capital of France?, Paris)
-- Multi turn: (What is the meaning of life, 42, Why is it 42?, That’s universe, ...)

3. Self curation (selecting high quality examples) using few shot prompting in addition to the prompt in Table 19 of the paper. Print out 5 examples of high quality examples and 5 examples of low quality examples. (25 points)
-   Push the dataset to HF hub and paste the url here

    https://huggingface.co/datasets/Justin8584/high_quality_dataset

-   Goal is to filter out bad samples
-   Method: using an LLM to rate the example --> LLM (meta/llama-7b-chat-hf): LLM(“Evaluate the quality of the instruction/response pair” + example.” Rate it from 1-5)

4. Finetune base model on dataset generated by step 3. Print out 5 example responses. (25 points)
-   Push the instruction fine tuned model to HF hub and paste the url here

    https://huggingface.co/Justin8584/fine_tuned_instruction



In [ ]:
!nvidia-smi

Wed Nov 20 03:05:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   52C    P8              17W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q datasets
!pip install --upgrade huggingface_hub datasets
!pip install -q --upgrade transformers datasets accelerate peft huggingface_hub

!pip install fsspec==2024.10.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.1.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.


In [ ]:
import bitsandbytes
import transformers

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn as nn
from torch.nn import functional as F
from torch import optim

## 1
Finetune the base language model (llama2 7B) with (output, instruction) pairs {(yi, xi)} from the seed data to obtain a backward model Myx := p(x|y). In other words, finetune a model that uses the output to predict the instruction. Use the openassistant-guanaco training set dataset. (25 points)


In [ ]:
from transformers import AutoTokenizer
model_tag = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_tag)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from datasets import load_dataset
dataset = load_dataset('timdettmers/openassistant-guanaco')
dataset = dataset['train'].train_test_split(test_size=0.1)
dataset = dataset.filter(lambda x: len(tokenizer.tokenize(x['text'])) < 256)

print(dataset)

alpaca_prompt = """Below is a response that appropriately completes the request. Write an instruction that describes a task.
### Response:
{}

### Instruction:
{}"""

EOS_TOKEN = tokenizer.eos_token

README.md:   0%|          | 0.00/395 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


openassistant_best_replies_train.jsonl:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

openassistant_best_replies_eval.jsonl:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8861 [00:00<?, ? examples/s]

Filter:   0%|          | 0/985 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3024
    })
    test: Dataset({
        features: ['text'],
        num_rows: 343
    })
})


In [ ]:
alpaca_prompt_M0 = """Below is an instruction that contains a task. Write a response that appropriately completes the request.
### Instruction:
{}

### Response:
{}"""

In [ ]:
splits = ['train', 'test']
texts = []
texts_M0=[]
for split in splits:
    for text in dataset[split]["text"]:
        chunks = text.split("### ")
        chunks = [c for c in chunks if len(c) > 0]
        chunks = chunks[:2]
        if not (chunks[0].startswith('Human: ') or chunks[1].startswith('Assistant: ')):
            continue
        instruction = chunks[0].replace("Human: ", "")
        response = chunks[1].replace("Assistant: ", "")

        texts.append(alpaca_prompt.format(response,instruction) + EOS_TOKEN)
        texts_M0.append(alpaca_prompt_M0.format(instruction,response) + EOS_TOKEN)


In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict({"text": texts})
dataset_M0= Dataset.from_dict({"text": texts_M0})

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 3367
})

In [ ]:
dataset_M0["text"][0]

'Below is an instruction that contains a task. Write a response that appropriately completes the request.\n### Instruction:\nEn que libros se basa el espiritismo de Allan Kardec?\n\n### Response:\nEl espiritismo de Allan Kardec se basa en varios libros que él mismo escribió y compiló, incluyendo:\n\n1. *El Libro de los Espíritus* (1857)\n2. *El Libro de los Médiums* (1861)\n3. *El Evangelio según el Espiritismo* (1864)\n4. *El Cielo y el Infierno* (1865)\n5. *La Génesis* (1868)\n\nEstos cinco libros son conocidos como "Los Cinco Libros de Allan Kardec" y constituyen la base del espiritismo kardeciano. En ellos, Kardec presenta sus enseñanzas sobre la comunicación con los espíritus, la reencarnación, la vida después de la muerte y la moralidad espiritual. Estos libros han sido traducidos a varios idiomas y son considerados como los textos fundamentales del espiritismo.</s>'

In [ ]:
max_length = 256
def generate_and_tokenize_prompt(prompt):
    result = tokenizer(
        prompt['text'],
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
tokenized_dataset = dataset.map(generate_and_tokenize_prompt)
tokenized_dataset_M0=dataset_M0.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/3367 [00:00<?, ? examples/s]

Map:   0%|          | 0/3367 [00:00<?, ? examples/s]

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
    model_tag,
    quantization_config=quant_config,
    device_map="auto",
)
model.config.use_cache = False
model.config.pretraining_tp = 1


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "fc1",
        "fc2",
        "dense",
        "lm_head"
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

### Train a backword model

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import os
os.environ["WANDB_MODE"] = "disabled"

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    lr_scheduler_type='cosine',
    max_steps=50,
    learning_rate=2e-5,
    optim="paged_adamw_8bit",
    logging_steps=5,
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
5,3.005500
10,2.809500
15,2.633800
20,3.036000
25,2.363000
30,2.518600
35,1.922700
40,2.050300
45,2.603900
50,2.423100


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=2.5366356277465822, metrics={'train_runtime': 25.2701, 'train_samples_per_second': 1.979, 'train_steps_per_second': 1.979, 'total_flos': 509465434521600.0, 'train_loss': 2.5366356277465822, 'epoch': 0.01485001485001485})

### Train a forward model

In [ ]:
training_args = TrainingArguments(
    output_dir="./output_1",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    lr_scheduler_type='cosine',
    max_steps=50,
    learning_rate=2e-5,
    optim="paged_adamw_8bit",
    logging_steps=5,
)
trainer_M0 = Trainer(
    model=model,
    train_dataset=tokenized_dataset_M0,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer_M0.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
5,2.466700
10,2.222900
15,2.103800
20,2.572200
25,1.946900
30,2.125300
35,1.571600
40,1.766500
45,1.978400
50,2.049000


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=50, training_loss=2.080311918258667, metrics={'train_runtime': 21.827, 'train_samples_per_second': 2.291, 'train_steps_per_second': 2.291, 'total_flos': 509465434521600.0, 'train_loss': 2.080311918258667, 'epoch': 0.01485001485001485})

In [ ]:
from huggingface_hub import notebook_login

# Use the notebook_login function to log in
notebook_login()

In [ ]:
new_model_name = "llama-2-7b-chat-guanaco"
new_model_name_fwd= "llama-2-7b-chat-guanaco_fwd"

trainer.processing_class = tokenizer
trainer_M0.processing_class = tokenizer

trainer.model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)
trainer.model.push_to_hub('Justin8584/backward_model')

trainer_M0.model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)
trainer_M0.model.push_to_hub('Justin8584/forward_model')

print(trainer.model)
print(trainer.processing_class)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/367M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/367M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

## 2
Self-Augmentation -- Randomly sample a subset of size 150 and generate instructions from the LIMA dataset’s completions and filtering out any mutli-turn examples. Print out 5 examples of generated instructions. (25 points)


In [ ]:
from datasets import load_dataset
import random

In [ ]:
lima_dataset = load_dataset('GAIR/lima')

print("LIMA Dataset Loaded:\n", lima_dataset)

README.md:   0%|          | 0.00/368 [00:00<?, ?B/s]

lima.py:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/27.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

LIMA Dataset Loaded:
 DatasetDict({
    train: Dataset({
        features: ['conversations', 'source'],
        num_rows: 1030
    })
    test: Dataset({
        features: ['conversations', 'source'],
        num_rows: 300
    })
})


In [ ]:
single_turn_lima = lima_dataset['train'].filter(lambda x: len(x['conversations']) == 2)

print("Filtered Single-Turn:\n", single_turn_lima)

sampled_lima = random.sample(single_turn_lima['conversations'], 150)

Filter:   0%|          | 0/1030 [00:00<?, ? examples/s]

Filtered Single-Turn:
 Dataset({
    features: ['conversations', 'source'],
    num_rows: 1000
})


In [ ]:
# Generate instructions using the backward model
instructions = []
max_input_length = 1500

for i, conv in enumerate(sampled_lima):

    # Extract the question and answer pair directly
    ques, ans = conv[0], conv[1]

    # Prepare the input prompt for the backward model
    prompt = f"Generate an instruction for the following output:\n{ans}\n generated question:"
    input_ids = trainer.processing_class.encode(prompt, return_tensors="pt")

    # Skip examples that exceed the max_input_length
    if len(input_ids[0]) > max_input_length:
        print(f"Skipping example {i+1} due to length: {len(input_ids[0])} tokens")
        continue

    # Move input_ids to the model's device (e.g., 'cuda' for GPU)
    input_ids = input_ids.to(trainer.model.device)

    # Generate the instruction using the backward model
    generated_ids = trainer.model.generate(input_ids, max_length=1500)
    generated_instruction = trainer.processing_class.decode(generated_ids[0], skip_special_tokens=True)

    # Store the generated instruction and original response
    instructions.append({
        "instruction": generated_instruction.split("generated question:")[-1].strip(),
        "response": ans
    })

    # Print progress every 10 examples
    if i % 10 == 0:
        print(f"Completed {i+1} predictions")

# Print out 5 examples of generated instructions
print("Sample of 5 Generated Instructions and Responses:")
for example in instructions[:5]:
    print("Generated Instruction:", example["instruction"])
    print("Response:", example["response"])
    print("\n")

Skipping example 1 due to length: 1960 tokens
Skipping example 11 due to length: 1906 tokens
Skipping example 12 due to length: 3607 tokens
Completed 21 predictions
Skipping example 29 due to length: 2429 tokens
Skipping example 30 due to length: 3381 tokens
Completed 31 predictions
Skipping example 35 due to length: 2670 tokens
Skipping example 40 due to length: 2112 tokens
Completed 41 predictions
Skipping example 49 due to length: 2853 tokens
Completed 51 predictions
Skipping example 58 due to length: 2009 tokens
Completed 61 predictions
Skipping example 62 due to length: 2547 tokens
Completed 71 predictions
Skipping example 72 due to length: 2163 tokens
Completed 81 predictions
Skipping example 83 due to length: 2417 tokens
Completed 91 predictions
Skipping example 93 due to length: 1622 tokens
Skipping example 100 due to length: 1617 tokens
Completed 101 predictions
Skipping example 104 due to length: 2993 tokens
Skipping example 106 due to length: 1643 tokens
Skipping example 110

In [ ]:
# For Instruction Clean up
# If further format adjustment is required, create a final list with cleaned instructions.
new_augmented_example = []
for example in instructions:
    new_example = {
        'instruction': example['instruction'],
        'output': example['response']
    }
    new_augmented_example.append(new_example)

# Display the first clean instruction for verification
print("Final Cleaned Instruction:", new_augmented_example[0]['instruction'])

Final Cleaned Instruction: What is the difference between using prop() and attr() in jQuery?

Answer:
In general, you should use ```prop()``` to get or set properties of an element, while you should use ```attr()``` to get or set attributes.

In the example provided, the ```prop()``` method is used to set the ```disabled``` property of an element, whereas the ```attr()``` method would set the ```disabled``` attribute directly.

In general, you should use ```prop()``` when you want to modify the dynamic state of an element, such as setting the value of a property. On the other hand, you should use ```attr()``` when you want to get or set an attribute of an element, such as the value of an attribute.

It is important to note that before jQuery 3.0, using ```removeAttr()```.


## 3
Self curation (selecting high quality examples) using few shot prompting in addition to the prompt in Table 1 of the paper. Print out 5 examples of high quality examples and 5 examples of low quality examples. (25 points)

- Push the dataset to HF hub and paste the url here
- Goal is to filter out bad samples
- Method: using an LLM to rate the example
LLM (meta/llama-7b-chat-hf): LLM(“Evaluate the quality of the instruction/response pair” + example.” Rate it from 1-5)


In [ ]:
from datasets import Dataset

In [ ]:
# Define the quality rating prompt
rating_prompt = """Below is an instruction from a user and a candidate answer. Evaluate whether the answer is a good example of how an AI Assistant should respond.
Please assign a score using the following 5-point scale:
1: Incomplete, vague, or irrelevant.
2: Partially addresses the user's question.
3: Helpful but not written by an AI Assistant.
4: Mostly complete, AI-focused response.
5: Perfect, high-quality AI response.

Please provide a brief reason for the rating, then write "Score: <rating>" on the last line.

Instruction: {}\n
Answer: {}
"""

In [ ]:
# Maximum number of examples to curate
max_iterations = 100
curated_set = []

# Evaluate each example and generate a score
for i, example in enumerate(new_augmented_example):
    if i >= max_iterations:
        break

    # Format the prompt with the instruction and response
    prompt_text = rating_prompt.format(example["instruction"], example["output"])
    input_ids = trainer_M0.processing_class.encode(prompt_text, return_tensors="pt")

    input_ids = input_ids.to(trainer_M0.model.device)

    # Generate the model's evaluation score
    generated_ids = trainer_M0.model.generate(input_ids, max_new_tokens=512)
    evaluation_output = trainer_M0.processing_class.decode(generated_ids[0], skip_special_tokens=True)

    # Append example with generated evaluation score
    curated_set.append({
        "generated_question": example["instruction"],
        "answer": example["output"],
        "score": evaluation_output
    })



In [ ]:
# Parse the model-generated score and filter high-quality examples
new_scores = []
for sample in curated_set:
    try:
        # Extract the numeric score from the model output
        score_text = sample["score"].rsplit("Score:", 1)[-1].strip(" <>")
        score = float(score_text[:1])

        new_scores.append({
            "generated_question": sample["generated_question"],
            "answer_to_question": sample["answer"],
            "score": score
        })
    except ValueError as ve:
        print(f"Error processing score for {sample}: {ve}")
        continue


# Filter scores based on quality threshold
high_ranking_score = float(3)
filtered_scores = [score for score in new_scores if score["score"] >= high_ranking_score]

Error processing score for {'generated_question': 'What is the difference between a default value and a fixed value in XML Schema?\n\nAnswer: A default value is a value that is assigned to an attribute when it is not specified, while a fixed value is a value that is fixed and cannot be changed.\n\n```\n```\n\ngenerated answer: A default value is a value that is assigned to an attribute when', 'answer': 'New, expanded answer to an old, commonly asked question...\nDefault Values\n\n* Occurrence constraints ```minOccurs``` and ```maxOccurs``` default to ```1```.\n\n\nCommon Cases Explained\n```\n```\nmeans ```A``` is required and must appear exactly once.\n\n```\n```\nmeans ```A``` is optional and may appear at most once.\n\n``` \n```\nmeans ```A``` is required and may repeat an unlimited number of times.\n\n``` \n```\nmeans ```A``` is optional and may repeat an unlimited number of times.\n\nSee Also\n\n* W3C XML Schema Part 0: Primer\n\n  In general, an element is required to appear when

In [ ]:
# Prepare and push dataset to Hugging Face Hub
texts_for_dataset = []
for data in filtered_scores:
    texts_for_dataset.append(alpaca_prompt_M0.format(data['generated_question'], data['answer_to_question']) + EOS_TOKEN)


dataset_to_push = Dataset.from_dict({"text": texts_for_dataset})
dataset_to_push.push_to_hub('Justin8584/high_quality_dataset')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/269 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Justin8584/high_quality_dataset/commit/790e1b1bf343a175ffdf43960c998ff14dc3921e', commit_message='Upload dataset', commit_description='', oid='790e1b1bf343a175ffdf43960c998ff14dc3921e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Justin8584/high_quality_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Justin8584/high_quality_dataset'), pr_revision=None, pr_num=None)

In [ ]:
# Print samples of high-quality and low-quality examples
print("\nSample High-Quality Examples (Score >= 3):")
for example in filtered_scores[:5]:  # Print first 5 high-quality examples
    print("Generated Question:", example["generated_question"])
    print("Answer:", example["answer_to_question"])
    print("Score:", example["score"])
    print("\n")

print("\nSample Low-Quality Examples (Score < 3):")
low_quality_examples = [score for score in new_scores if score["score"] < high_ranking_score]

# Print first 5 low-quality examples
for example in low_quality_examples[:5]:
    print("Generated Question:", example["generated_question"])
    print("Answer:", example["answer_to_question"])
    print("Score:", example["score"])
    print("\n")


Sample High-Quality Examples (Score >= 3):
Generated Question: What is the difference between using prop() and attr() in jQuery?

Answer:
In general, you should use ```prop()``` to get or set properties of an element, while you should use ```attr()``` to get or set attributes.

In the example provided, the ```prop()``` method is used to set the ```disabled``` property of an element, whereas the ```attr()``` method would set the ```disabled``` attribute directly.

In general, you should use ```prop()``` when you want to modify the dynamic state of an element, such as setting the value of a property. On the other hand, you should use ```attr()``` when you want to get or set an attribute of an element, such as the value of an attribute.

It is important to note that before jQuery 3.0, using ```removeAttr()```.
Answer: Always use the ```prop()``` method to enable or disable elements when using jQuery (see below for why).
In your case, it would be:
```$(&quot;#edit&quot;).click(function(ev

## 4
Finetune base model on dataset generated by step 3. Print out 5 example responses. (25 points)
- Push the instruction fine tuned model to HF hub and paste the url here


In [ ]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [ ]:
# Load the high-quality dataset from Hugging Face Hub
dataset = load_dataset('Justin8584/high_quality_dataset')
dataset = dataset['train'].train_test_split(test_size=0.1)
print("Dataset loaded and split:", dataset)

# Define end-of-sequence token and add it to each text example
EOS_TOKEN = "<|endoftext|>"
texts = []
for split in ['train', 'test']:
    for text in dataset[split]["text"]:
        texts.append(text + EOS_TOKEN)

# Recreate the dataset with appended EOS tokens
dataset = Dataset.from_dict({"text": texts})
max_length = 256

# Initialize the tokenizer
model_tag = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_tag)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

README.md:   0%|          | 0.00/269 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/77 [00:00<?, ? examples/s]

Dataset loaded and split: DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 69
    })
    test: Dataset({
        features: ['text'],
        num_rows: 8
    })
})


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Tokenize the dataset
def generate_and_tokenize_prompt(prompt):
    result = tokenizer(
        prompt['text'],
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

# Tokenize the dataset
tokenized_dataset = dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/77 [00:00<?, ? examples/s]

In [ ]:
# Configure 4-bit quantization
compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# Load the base model with quantization
model_with_new_dataset = AutoModelForCausalLM.from_pretrained(
    model_tag,
    quantization_config=quant_config,
    device_map="auto",
)
model_with_new_dataset.config.use_cache = False
model_with_new_dataset.config.pretraining_tp = 1

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# Apply LoRA for efficient fine-tuning
config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "fc1", "fc2", "dense", "lm_head"
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model_with_new_dataset = get_peft_model(model_with_new_dataset, config)

In [ ]:
# Define training arguments and initialize the Trainer
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    lr_scheduler_type='cosine',
    max_steps=50,
    learning_rate=2e-5,
    optim="paged_adamw_8bit",
    logging_steps=5,
)

trainer_new = Trainer(
    model=model_with_new_dataset,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# Fine-tune the model
trainer_new.train()

# Push the fine-tuned model to Hugging Face Hub
trainer_new.model.push_to_hub('Justin8584/fine_tuned_instruction')

# Test the fine-tuned model with 5 example prompts and print the responses
example_prompts = [
    "Explain the process of photosynthesis.",
    "What is the capital of Japan?",
    "How does an AI language model work?",
    "Describe the structure of a water molecule.",
    "What are the benefits of learning Python programming?"
]

for prompt in example_prompts:
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model_with_new_dataset.device)
    output_ids = model_with_new_dataset.generate(input_ids, max_length=100)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(f"Prompt: {prompt}")
    print(f"Response: {response}\n")

Step,Training Loss
5,2.388900
10,1.976200
15,1.840100
20,1.835600
25,1.646500
30,2.038300
35,1.359100
40,1.923400
45,1.134700
50,1.277900


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/367M [00:00<?, ?B/s]

Prompt: Explain the process of photosynthesis.
Response: Explain the process of photosynthesis. Einzelnen.
Photosynthesis is a complex process that involves several steps and requires energy from light, water, and carbon dioxide. Here are the main steps of photosynthesis:

1. Light-dependent reactions: In the first stage of photosynthesis, light energy from the sun is absorbed by pigments such as chlorophyll in the leaves of plants. This energy is used to excite

Prompt: What is the capital of Japan?
Response: What is the capital of Japan?
 Unterscheidung between Japanese and Japanese people.
The word "Japanese" can refer to the language, culture, or people of Japan. "Japanese" is often used to refer to the language and culture, while "Japanese people" or "Nihonjin" is used to refer to the people of Japan.

The capital of Japan is Tokyo. Tokyo is the political, economic, and cultural center of Japan and is

Prompt: How does an AI language model work?
Response: How does an AI language m